In [19]:
import tensorflow as tf
import os
# Choosing which GPU txhis notebook can access
# (useful when running multiple experiments in parallel, on different GPUs):
os.environ["CUDA_VISIBLE_DEVICES"]= "2" 

## 准备数据

_CIFAR-10_ 和 _CIFAR-100_ 数据集是著名的分类数据集。

第一个版本包含来自10个不同类的 60000张，32 x 32 图像。

第二个版本包含来自100个不同类的60000 32x  32 图像。

这么我么将使用 _CIFAR-100_，构建训练模型



***注意:*** 虽然这个分类任务比我们之前的 _MNIST_ 实验更复杂,

其复杂性并不等同于 _ImageNet_  的挑战，例如，

在这一章中提到，我们选择了 _CIFAR-100_，因为这个数据集很容易获取，并且足够轻，读者能够重现我们的实验。

### TensorFlow数据集

与其他数据集一样，_CIFAR-100_  也经常被用来研究他们的算法，（虽然 *ImageNet* 更有挑战性）或者优先考虑 *Open-Images*

为了帮助研究人员和业余爱好者，TensorFlow团队推出了一个可选的软件包:  **`tensorflow_datasets`** ([https://www.tensorflow.org/datasets](https://www.tensorflow.org/datasets))
该软件包提供了可下载的帮助函数和一些常用数据集


安装命令
```bash
# !pip install tensorflow-datasets
```

In [20]:
import tensorflow_datasets as tfds

# tfds.list_builders()


A detailed list can also be found on their website: [https://www.tensorflow.org/datasets/datasets](https://www.tensorflow.org/datasets/datasets).

我们选择 `"cifar100"`, 下载该模块, 以及显示详情:

In [21]:
cifar_builder = tfds.builder("cifar100")
cifar_builder.download_and_prepare()

print(cifar_builder.info)


tfds.core.DatasetInfo(
    name='cifar100',
    full_name='cifar100/3.0.2',
    description="""
    This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_path='/Users/theone/tensorflow_datasets/cifar100/3.0.2',
    download_size=160.71 MiB,
    dataset_size=132.03 MiB,
    features=FeaturesDict({
        'coarse_label': ClassLabel(shape=(), dtype=tf.int64, num_classes=20),
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=100),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling

TensorFlow数据集为我们提供了一些有用的信息，比如样本数、类型和大小、数据集描述等。


我们还可以访问类名，如下所示：

In [22]:
# 标签名称
print(cifar_builder.info.features["label"].names)


# “苹果”、“水族馆鱼”、“婴儿”、“熊”、“海狸”、“床”、“蜜蜂”、“甲虫”、“自行车”、“瓶子”、“碗”、“男孩”、“桥”、“公共汽车”、“蝴蝶”、“骆驼”、“罐头”、“城堡”、“毛毛虫”、“牛”、“椅子”、“黑猩猩”、“时钟”、“云”、“蟑螂”、“沙发”、“螃蟹”、“鳄鱼”、“杯子”、“恐龙”、“海豚”、“大象”、“比目鱼”、“森林”、“狐狸”，“女孩”、“仓鼠”、“房子”、“袋鼠”、“键盘”、“灯”、“割草机”、“豹子”、“狮子”、“蜥蜴”、“龙虾”、“男人”、“枫树”、“摩托车”、“山”、“老鼠”、“蘑菇”、“橡树”、“橘子”、“兰花”、“水獭”、“棕榈树”、“梨”、“皮卡车”、“松树”、“平原”、“盘子”、“罂粟”、“豪猪”、“负鼠”、“兔子”、“浣熊”，“雷”、“路”、“火箭”、“玫瑰”、“海”、“海豹”、“鲨鱼”、“悍妇”、“臭鼬”、“摩天大楼”、“蜗牛”、“蛇”、“蜘蛛”、“松鼠”、“电车”、“向日葵”、“甜椒”、“桌子”、“坦克”、“电话”、“电视”、“老虎”、“拖拉机”、“火车”、“鳟鱼”、“郁金香”、“海龟”、“衣橱”、“鲸鱼”、“柳树”、“狼”、“女人”、“蠕虫”

['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree',

In [23]:
#  粗糙的标签
print(cifar_builder.info.features["coarse_label"].names)

# 蚯蚓水生哺乳动物、鱼类、花卉、食品容器、水果和蔬菜、家用电器、家用家具、昆虫、大型食肉动物、大型人造户外物品、大型自然户外场景、大型杂食动物和食草动物、中型哺乳动物、非昆虫无脊椎动物、人、爬行动物、，“小型哺乳动物”、“树木”、“车辆1”、“车辆2”


['aquatic_mammals', 'fish', 'flowers', 'food_containers', 'fruit_and_vegetables', 'household_electrical_devices', 'household_furniture', 'insects', 'large_carnivores', 'large_man-made_outdoor_things', 'large_natural_outdoor_scenes', 'large_omnivores_and_herbivores', 'medium_mammals', 'non-insect_invertebrates', 'people', 'reptiles', 'small_mammals', 'trees', 'vehicles_1', 'vehicles_2']



我们将继续使用更复杂的100个类。

## 输入管道准备

我们有可用的续联和测试数据。现在，我们可以明确定义我们将培训的模型的输入管道：

In [24]:
import math

# 一些超参数:
input_shape = [224, 224, 3]  # We will resize the input images to this shape
# Images per batch (reduce/increase according to the machine's capability)
batch_size = 32
num_epochs = 300           # Max number of training epochs

# 训练/验证 Datasets:
train_cifar_dataset = cifar_builder.as_dataset(split=tfds.Split.TRAIN)
val_cifar_dataset = cifar_builder.as_dataset(split=tfds.Split.TEST)

# 分类总数:
num_classes = cifar_builder.info.features['label'].num_classes

#图片的数量:
num_train_imgs = cifar_builder.info.splits['train'].num_examples
num_val_imgs = cifar_builder.info.splits['test'].num_examples


In [25]:
print('训练数据实例: {}'.format(train_cifar_dataset))

# 预处理数据结构


训练数据实例: <PrefetchDataset shapes: {coarse_label: (), id: (), image: (32, 32, 3), label: ()}, types: {coarse_label: tf.int64, id: tf.string, image: tf.uint8, label: tf.int64}>


数据由 TensorFlow-Datasets 模块作为一个 **`tf.data.Dataset`** 实例。


第7章部分介绍了 **`tf.data`** API和高效输入管道的创建，所以我们在这里不做太多详细介绍。我们只会提到这一点。数据Dataset实例的行为类似于优化的数据迭代器。他们还拥有多种数据预处理方法。


In [26]:
train_cifar_dataset = train_cifar_dataset.repeat(num_epochs).shuffle(10000)


我们还希望对数据本身进行预处理，调整图像大小，忽略较粗糙的标签，并选择性地对图像应用一些随机增强（随机操作以人为地增加我们的模型可以训练的不同图像的数量——更多细节见第7章）。


因此，我们实现了一个函数，对样本进行相应的预处理，并将其传递给 `tf.data.Dataset` 数据集实例：

In [27]:
def _prepare_data_fn(features, input_shape, augment=False):
    """
    将图像大小调整为预期尺寸，然后选择。应用一些随机变换
    :param features:    Data
    :param input_shape: Shape expected by the models (images will be resized accordingly)
    :param augment:     标记以对图像应用一些随机增强
    :return:            增强图像, Labels
    """
    input_shape = tf.convert_to_tensor(input_shape)

    # Tensorflow数据集以特征字典的形式返回批次，这是估计器所期望的.
    # 为了训练Keras模型，以元组形式返回批处理内容更简单:
    image = features['image']
    label = features['label']
    # 将图像转换为浮动类型, 等比例缩放将 [0, 255] 转换到 [0., 1.]:
    image = tf.image.convert_image_dtype(image, tf.float32)

    if augment:
        # 随机水平翻转:
        image = tf.image.random_flip_left_right(image)

        # 随机亮度调整
        image = tf.image.random_brightness(image, max_delta=0.1)

        # 随机饱和调整 
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        # 控制像素值 ？？
        image = tf.clip_by_value(image, 0.0, 1.0)

        # 随机调整大小并随机裁剪回预期大小:

        random_scale_factor = tf.random.uniform(
            [1], minval=1., maxval=1.4, dtype=tf.float32)
        scaled_height = tf.cast(tf.cast(input_shape[0], tf.float32) * random_scale_factor,
                                tf.int32)
        scaled_width = tf.cast(tf.cast(input_shape[1], tf.float32) * random_scale_factor,
                               tf.int32)
        scaled_shape = tf.squeeze(tf.stack([scaled_height, scaled_width]))
        image = tf.image.resize(image, scaled_shape)
        image = tf.image.random_crop(image, input_shape)
    else:
        image = tf.image.resize(image, input_shape[:2])

    return image, label


In [28]:
import functools

prepare_data_fn_for_train = functools.partial(_prepare_data_fn,
                                              input_shape=input_shape,
                                              augment=True)

train_cifar_dataset = train_cifar_dataset.map(
    prepare_data_fn_for_train, num_parallel_calls=4)

# We also ask the dataset to batch the samples:
train_cifar_dataset = train_cifar_dataset.batch(batch_size)

train_cifar_dataset = train_cifar_dataset.prefetch(
    1)  # improve time performance c.f. Chapter 7


In [29]:
print(val_cifar_dataset)

<PrefetchDataset shapes: {coarse_label: (), id: (), image: (32, 32, 3), label: ()}, types: {coarse_label: tf.int64, id: tf.string, image: tf.uint8, label: tf.int64}>


类似地，我们进一步准备了验证数据集（尽管没有洗牌或扩充它）：

In [30]:
prepare_data_fn_for_val = functools.partial(_prepare_data_fn,
                                            input_shape=input_shape,
                                            augment=False)

val_cifar_dataset = (val_cifar_dataset
                     .repeat()
                     .map(prepare_data_fn_for_val, num_parallel_calls=4)
                     .batch(batch_size)
                     .prefetch(1))


## 用Keras实现ResNet



In [31]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Activation, Dense, Flatten, Conv2D, MaxPooling2D,
    GlobalAveragePooling2D, AveragePooling2D, BatchNormalization, add)
import tensorflow.keras.regularizers as regulizers


### 实现残差网路模块


为了实现这样一个模型生成器功能，能够生成各种大小的网络，我们需要将ResNet实现模块化。因此，我们将以模块化的方式实现ResNet的主要组件，即剩余块，以便我们可以无限期地将它们链接起来。



在第4章中，我们看到剩余块由以下部分组成：

-一个 _residual_ 分支，对输入应用3x3卷积，然后进行批量标准化和重新激活；

-一个 _shortcut_ 分支，直接转发输入，无需修改，或仅应用1x1卷积以适应输入量，以防另一个分支中的输入量发生变化；

-一个  _merge_  u操作，在两个分支的结果之间执行元素级加法。 



现在我们将实现每个子模块。首先，让我们将  `conv-batnchnorm-relu`   堆栈打包成一个可以随意重用的函数：

In [32]:
def _res_conv(filters, kernel_size=3, padding='same', strides=1, use_relu=True, use_bias=False, name='cbr',
              kernel_initializer='he_normal', kernel_regularizer=regulizers.l2(1e-4)):
    """
    Return a layer block chaining conv, batchnrom and reLU activation.
    :param filters:                 Number of filters.
    :param kernel_size:             Kernel size.
    :param padding:                 Convolution padding.
    :param strides:                 Convolution strides.
    :param use_relu:                Flag to apply ReLu activation at the end.
    :param use_bias:                Flag to use bias or not in Conv layer.
    :param name:                    Name suffix for the layers.
    :param kernel_initializer:      Kernel initialisation method name.
    :param kernel_regularizer:      Kernel regularizer.
    :return:                        Callable layer block
    """

    def layer_fn(x):
        conv = Conv2D(
            filters=filters, kernel_size=kernel_size, padding=padding, strides=strides, use_bias=use_bias,
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer,
            name=name + '_c')(x)
        res = BatchNormalization(axis=-1, name=name + '_bn')(conv)
        if use_relu:
            res = Activation("relu", name=name + '_r')(res)
        return res

    return layer_fn
